In [2]:
import cv2
import numpy as np
import pyfirmata
import threading
import time
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from filterpy.kalman import KalmanFilter

# Kết nối Arduino
port = "COM3"
board = pyfirmata.Arduino(port)
servo_pinX = board.get_pin('d:9:s')
servo_pinY = board.get_pin('d:10:s')

# Load YOLO model
model = YOLO("model.pt")

# Deep SORT Tracker tối ưu
tracker = DeepSort(max_age=150, max_iou_distance=0.4, nn_budget=100)

# Kalman Filter
kf = KalmanFilter(dim_x=4, dim_z=2)
kf.F = np.array([[1, 1, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 1, 1],
                 [0, 0, 0, 1]])
kf.H = np.array([[1, 0, 0, 0],
                 [0, 0, 1, 0]])
kf.P *= 1000
kf.x = np.array([0, 0, 0, 0])

ws, hs = 1280, 720
selected_id = None

# Camera
cap = cv2.VideoCapture(1)

# Quản lý track đã mất
lost_tracks = defaultdict(lambda: {"last_seen": 0, "track": None})
LOST_TIMEOUT = 5  # seconds

# Lưu các phát hiện mới nhất
detections = []
latest_frame = None
yolo_lock = threading.Lock()


def get_servo_range_by_distance(distance_m):
    if distance_m > 2:
        return (0, 180)
    elif distance_m > 1:
        return (30, 150)
    else:
        return (60, 120)


def estimate_distance(pixel_width):
    REAL_WIDTH = 0.5  # meters
    FOCAL_LENGTH = 700  # pixels
    if pixel_width == 0:
        return -1
    return (FOCAL_LENGTH * REAL_WIDTH) / pixel_width


def run_yolo_loop():
    global detections, latest_frame
    while True:
        if latest_frame is not None:
            with yolo_lock:
                frame_copy = latest_frame.copy()
            results = model(frame_copy)
            new_detections = []
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0].item()
                class_id = int(box.cls[0].item())
                if class_id == 0:
                    new_detections.append(([x1, y1, x2 - x1, y2 - y1], confidence, class_id))
            detections = new_detections


# Khởi động YOLO thread
threading.Thread(target=run_yolo_loop, daemon=True).start()

# Chuột chọn đối tượng
cv2.namedWindow("Tracking")
def select_object(event, x, y, flags, param):
    global selected_id
    if event == cv2.EVENT_LBUTTONDOWN:
        for track in param:
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            if x1 <= x <= x2 and y1 <= y <= y2:
                selected_id = track.track_id
                print(f"Chọn đối tượng ID: {selected_id}")
cv2.setMouseCallback("Tracking", select_object, param=[])

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    with yolo_lock:
        latest_frame = frame.copy()

    current_time = time.time()

    # Cập nhật tracker
    tracks = tracker.update_tracks(detections, frame=frame)
    cv2.setMouseCallback("Tracking", select_object, param=tracks)

    active_ids = set()

    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue

        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        active_ids.add(track_id)

        if track_id in lost_tracks:
            del lost_tracks[track_id]

        if selected_id is not None and track_id == selected_id:
            kf.predict()
            kf.update([cx, cy])
            predicted = kf.x[:2]
            pixel_width = x2 - x1
            distance_m = estimate_distance(pixel_width)
            servo_min, servo_max = get_servo_range_by_distance(distance_m)
            servo_x = np.clip(np.interp(predicted[0], [0, ws], [servo_max, servo_min]), servo_min, servo_max)
            servo_y = np.clip(np.interp(predicted[1], [0, hs], [0, 180]), 0, 180)
            servo_pinX.write(servo_x)
            servo_pinY.write(servo_y)

        color = (0, 255, 0) if track_id == selected_id else (255, 0, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        label = f"ID {track_id}"
        if selected_id == track_id:
            label += f" X: {servo_x:.0f}, Y: {servo_y:.0f}"
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Xử lý track đã mất
    for track in tracker.tracker.tracks:
        track_id = track.track_id
        if track_id not in active_ids and track_id == selected_id:
            if track_id not in lost_tracks:
                lost_tracks[track_id] = {"last_seen": current_time, "track": track}
            elif current_time - lost_tracks[track_id]["last_seen"] > LOST_TIMEOUT:
                print(f"Hủy theo dõi ID {track_id} sau 5s mất dấu.")
                selected_id = None
                del lost_tracks[track_id]

    # Vẽ khung cho các đối tượng đã mất (ghost box)
    for track_id, data in lost_tracks.items():
        if current_time - data["last_seen"] <= LOST_TIMEOUT:
            track = data["track"]
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 1)
            cv2.putText(frame, f"ID {track_id} (ghost)", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    cv2.imshow("Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


SerialException: could not open port 'COM3': PermissionError(13, 'Access is denied.', None, 5)